In [82]:
!pip install transformers

In [68]:
from transformers import pipeline
# this classifier is not working well at all, returns same probabilities for all classes.
classifier = pipeline("zero-shot-classification",
                      model="gsarti/scibert-nli")

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at gsarti/scibert-nli and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/135 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/217k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [83]:
import pandas as pd
url = 'https://raw.githubusercontent.com/nasa-petal/petal-labeler-data-pipeline/master/FinalFile/golden.json'
df = pd.read_json(url, orient='columns')
df.head(10)

,paper,mag,venue_mag,author,reference,title,abstract,petalID,doi,venue,level1,level2,level3,url,isBiomimicry,fullDocLink,isOpenAccess
0,2103410568,"['bubble nest', 'nest', 'mixing', 'bubble', 'p...",['Biology Letters'],"[2346835213, 2098042950]","[2130285640, 2066345165, 2054319467, 204771406...","['building', 'home', 'foam', 'tungara', 'frog'...","['frogs', 'build', 'foam', 'nests', 'floating'...",0,10.1098/RSBL.2009.0934,"[""Weird Nature: An Astonishing Exploration of ...","['physically_assemble/disassemble', 'protect_f...","['physically_assemble_structure', 'protect_fro...","['protect_from_animals', 'protect_from_loss_of...",https://royalsocietypublishing.org/doi/10.1098...,Y,https://royalsocietypublishing.org/doi/10.1098...,True
1,2138292607,"['sunset', 'earth s magnetic field', 'compass'...",['Proceedings of the National Academy of Scien...,"[2132083079, 2425702268, 2552946098]","[1493129647, 2037761037, 1984592609, 213699427...","['nocturnal', 'mammal', 'greater', 'mouse', 'e...","['evidence', 'suggests', 'bats', 'detect', 'ge...",1,10.1073/PNAS.0912477107,['Proceedings of the National Academy of Scien...,['sense_send_or_process_information'],['sense_signals/environmental_cues'],['sense_spatial_awareness/balance/orientation'],https://www.pnas.org/content/107/15/6941,Y,https://www.pnas.org/content/107/15/6941.full.pdf,True
2,2005539166,"['sepia mestus', 'optomotor response', 'cuttle...",['The Journal of Experimental Biology'],"[2163942483, 3088803717]","[2035108601, 2155571491, 2159857711, 207521876...","['polarization', 'sensitivity', 'two', 'specie...","['existence', 'polarization', 'sensitivity', '...",2,10.1242/JEB.042937,"['The Journal of Experimental Biology', 'Curre...",['sense_send_or_process_information'],['sense_signals/environmental_cues'],"['sense_light_in_the_non-visible_spectrum', 's...",https://jeb.biologists.org/content/213/19/3364,Y,https://journals.biologists.com/jeb/article-pd...,True
3,2151557512,"['energy source', 'cellulase', 'archaea', 'cel...",['Nature Communications'],"[3146712807, 3146712807, 2189911855, 265906908...","[2141885858, 2120772351, 2108929776, 212024499...","['identification', 'characterization', 'multid...","['archaea', 'microorganisms', 'wide', 'range',...",3,10.1038/NCOMMS1373,['Nature Communications'],"['chemically_modify_or_change_energy_state', '...","['chemically_break_down', 'protect_from_non-li...","['chemically_break_down_organic_compounds', 'p...",https://www.nature.com/articles/ncomms1373,Y,https://www.nature.com/articles/ncomms1373.pdf,True
4,2160542693,"['galactan', 'coralline algae', 'cell wall', '...",['Journal of Phycology'],"[1790079306, 2557664461, 189902495, 2140516860]","[2001396309, 2057007315, 1968834637, 212515227...","['differences', 'polysaccharide', 'structure',...","['articulated', 'coralline', 'calliarthron', '...",4,10.1111/J.1529-8817.2010.00828.X,"['The Journal of Experimental Biology', 'Natur...",['maintain_structural_integrity'],['manage_structural_forces'],"['manage_shear', 'manage_stress/strain']",https://onlinelibrary.wiley.com/doi/abs/10.111...,Y,,False
5,2130285640,"['nest', 'peptide sequence', 'insect', 'cystat...",['Proceedings of The Royal Society B: Biologic...,"[2140984283, 2490734702, 2116739833, 2098042950]","[1975304761, 2089047063, 2034767282, 210406370...","['foam', 'nest', 'components', 'tungara', 'fro...","['foam', 'nests', 'tungara', 'frog', 'engystom...",5,10.1098/RSPB.2008.1939,"[""Weird Nature: An Astonishing Exploration of ...","['maintain_structural_integrity', 'protect_fro...","['manage_structural_forces', 'protect_from_liv...","['manage_impact', 'protect_from_animals', 'pro...",https://royalsocietypublishing.org/doi/10.1098...,Y,https://royalsocietypublishing.org/doi/10.1098...,True
6,2168114966,"['eremobatidae', 'suctorial', 'eremobates', 's...",['Arthropod Structure & Development'],"[2907238235, 2075816131, 1452472624, 215895902...","[1676421722, 2102501940, 2242001249, 209572581...","['suctorial', 'organ', 'solifugae', 'ara

In [89]:
# remap level1 labels
'''
df.level1 = df.level1.replace({'physically_assemble/disassemble' : 'physically assembling or disassembling',
                               'protect_from_harm':'protecting',
                               'sense_send_or_process_information':'sensing, sending or processing information',
                               'sense,_send,_or_process_information': 'sensing, sending or processing information',
                               'chemically_modify_or_change_energy_state':'chemically assembling or breaking down',
                               'maintain_structural_integrity':'managing mechanical forces',
                               'attach':'attaching',
                               'move':'movement',
                               'process_resources':'manipulating solids, liquids, gases or energy',
                               'sustain_ecological_community':'sustaining ecological community',
                               'maintain_ecological_community':'sustaining ecological community',
                               'manipulate_solids,_liquids,_gases,_or_energy':'manipulating solids, liquids, gases or energy',
                               'change_size_or_color':''}, regex=True)
'''
df.level1 = df.level1.replace({'physically_assemble/disassemble' : 'physically_assemble_or_disassemble',
                               'sense,_send,_or_process_information': 'sense_send_or_process_information',
                               'maintain_ecological_community':'sustain_ecological_community',
                               'manipulate_solids,_liquids,_gases,_or_energy':'manipulate_solids_liquids_gases_or_energy'}, regex=True)

In [57]:
from ast import literal_eval
df['level1'] = df['level1'].apply(literal_eval)

# show unique level1 labels have to 
from itertools import chain

unique_lists_in_items = df.level1
set_of_items = set(chain(*unique_lists_in_items))
print(set_of_items)

{'', 'sensing, sending or processing information', 'sustaining ecological community', 'movement', 'attaching', 'physically assembling or disassembling', 'managing mechanical forces', 'manipulating solids, liquids, gases or energy', 'chemically assembling or breaking down', 'protecting'}


In [90]:
# remove rows with no level1 labels
df = df[df.level1 != '[]']
df = df[df.level1 != "['']"]

In [ ]:
for index, row in df.iterrows():
    print(row['c1'], row['c2'])

In [102]:
sequence_to_classify = "Evidence for self-cleaning in gecko setae. A tokay gecko can cling to virtually any surface and support its body mass with a single toe by using the millions of keratinous setae on its toe pads. Each seta branches into hundreds of 200-nm spatulae that make intimate contact with a variety of surface profiles. We showed previously that the combined surface area of billions of spatulae maximizes van der Waals interactions to generate large adhesive and shear forces. Geckos are not known to groom their feet yet retain their stickiness for months between molts. How geckos manage to keep their feet clean while walking about with sticky toes has remained a puzzle until now. Although self-cleaning by water droplets occurs in plant and animal surfaces, no adhesive has been shown to self-clean. In the present study, we demonstrate that gecko setae are a self-cleaning adhesive. Geckos with dirty feet recovered their ability to cling to vertical surfaces after only a few steps. Self-cleaning occurred in arrays of setae isolated from the gecko. Contact mechanical models suggest that self-cleaning occurs by an energetic disequilibrium between the adhesive forces attracting a dirt particle to the substrate and those attracting the same particle to one or more spatulae. We propose that the property of self-cleaning is intrinsic to the setal nanostructure and therefore should be replicable in synthetic adhesive materials in the future."
sequence_to_classify2 = "The plant surface is by default flat, and development away from this default is thought to have some function of evolutionary advantage. Although the functions of many plant epidermal cells have been described, the function of conical epidermal cells, a defining feature of petals in the majority of insect-pollinated flowers, has not. The location and frequency of conical cells have led to speculation that they play a role in attracting animal pollinators. Snapdragon (Antirrhinum) mutants lacking conical cells have been shown to be discriminated against by foraging bumblebees. Here we investigated the extent to which a difference in petal surface structure influences pollinator behavior through touch-based discrimination. To isolate touch-based responses, we used both biomimetic replicas of petal surfaces and isogenic Antirrhinum lines differing only in petal epidermal cell shape. We show that foraging bumblebees are able to discriminate between different surfaces via tactile cues alone. We find that bumblebees use color cues to discriminate against flowers that lack conical cells--but only when flower surfaces are presented at steep angles, making them difficult to manipulate. This facilitation of physical handling is a likely explanation for the prevalence of conical epidermal petal cells in most flowering plants."
sequence_to_classify3 = "In stark contrast to most aggressive predators, Dosidicus gigas (jumbo squids) do not use minerals in their powerful mouthparts known as beaks. Their beaks instead consist of a highly sclerotized chitinous composite with incremental hydration from the tip to the base. We previously reported l-3,4-dihydroxyphenylalanine (dopa)-histidine (dopa-His) as an important covalent cross-link providing mechanical strengthening to the beak material. Here, we present a more complete characterization of the sclerotization chemistry and describe additional cross-links from D. gigas beak. All cross-links presented in this report share common building blocks, a family of di-, tri-, and tetra-histidine-catecholic adducts, that were separated by affinity chromatography and high performance liquid chromatography (HPLC) and identified by tandem mass spectroscopy and proton nuclear magnetic resonance (1H NMR). The data provide additional insights into the unusually high cross-link density found in mature beaks. Furthermore, we propose both a low molecular weight catechol, and peptidyl-dopa, to be sclerotization agents of squid beak. This appears to represent a new strategy for forming hard tissue in animals. The interplay between covalent cross-linking and dehydration on the graded properties of the beaks is discussed."
sequence_to_classify4 = "As human activities continue to alter the global nitrogen cycle, the ability to predict the impact of increased nitrogen loading to freshwater systems is becoming more and more important. Nitrogen retention is of particular interest because it is through its combined processes (denitrification, nitrogen sedimentation and uptake by aquatic plants) that local and downstream nitrogen concentrations are reduced. Here, we compare the magnitude of nitrogen retention and its components in wetlands, lakes and rivers. We show that wetlands retain the highest proportion of total nitrogen loading, followed by lakes and then rivers. The differences in the proportion of N retained among systems is explained almost entirely by differences in water discharge. Denitrification is the primary mechanism of nitrogen retention, followed by nitrogen sedimentation and uptake by aquatic plants."
sequence_to_classify5 = "Cooling is a significant end-use of energy globally and a major driver of peak electricity demand. Air conditioning, for example, accounts for nearly fifteen per cent of the primary energy used by buildings in the United States. A passive cooling strategy that cools without any electricity input could therefore have a significant impact on global energy consumption. To achieve cooling one needs to be able to reach and maintain a temperature below that of the ambient air. At night, passive cooling below ambient air temperature has been demonstrated using a technique known as radiative cooling, in which a device exposed to the sky is used to radiate heat to outer space through a transparency window in the atmosphere between 8 and 13¬†micrometres. Peak cooling demand, however, occurs during the daytime. Daytime radiative cooling to a temperature below ambient of a surface under direct sunlight has not been achieved because sky access during the day results in heating of the radiative cooler by the Sun. Here, we experimentally demonstrate radiative cooling to nearly 5¬†degrees Celsius below the ambient air temperature under direct sunlight. Using a thermal photonic approach, we introduce an integrated photonic solar reflector and thermal emitter consisting of seven layers of HfO2 and SiO2 that reflects 97 per cent of incident sunlight while emitting strongly and selectively in the atmospheric transparency window. When exposed to direct sunlight exceeding 850 watts per square metre on a rooftop, the photonic radiative cooler cools to 4.9¬†degrees Celsius below ambient air temperature, and has a cooling power of 40.1 watts per square metre at ambient air temperature. These results demonstrate that a tailored, photonic approach can fundamentally enable new technological possibilities for energy efficiency. Further, the cold darkness of the Universe can be used as a renewable thermodynamic resource, even during the hottest hours of the day."
sequence_to_classify6 = "Autotomy refers to the voluntary shedding of a body part; a renowned example is tail loss among lizards as a response to attempted predation. Although many aspects of lizard tail autotomy have been studied, the detailed morphology and mechanism remains unclear. In the present study, we showed that tail shedding by the Tokay gecko (Gekko gecko) and the associated extracellular matrix (ECM) rupture were independent of proteolysis. Instead, lizard caudal autotomy relied on biological adhesion facilitated by surface microstructures. Results based on bio-imaging techniques demonstrated that the tail of Gekko gecko was pre-severed at distinct sites and that its structural integrity depended on the adhesion between these segments."
candidate_labels = ['Attach', 'Attach permanently', 'Attach temporarily', 'Modify color or camouflage', 'Change structural color', 'Change chemical color or pigmentation', 'Camouflage or mimicry', 'Modify size shape or material properties', 'Change size or shape', 'Change material properties', 'Modify or convert energy', 'Modify or convert electrical energy', 'Modify or convert magnetic energy', 'Modify or convert chemical energy', 'Modify or convert mechanical energy', 'Modify or convert thermal energy', 'Modify or convert light energy', 'Assemble or break down structure', 'Physically assemble structure', 'Break down structure', 'Move on or through solids liquids or gases', 'Passive movement', 'Passively move through or on solids', 'Passively move through or on liquids', 'Passively move through gases', 'Passively move through or on granular media', 'Active movement', 'Actively move through or on solids', 'Actively move through or on liquids', 'Actively move through gases', 'Actively move through or on granular media', 'Protect from living or non-living threats', 'Protect from living threats', 'Protect from animals', 'Protect from plants', 'Protect from fungi', 'Protect from microbes', 'Protect from non-living threats', 'Protect from solids', 'Protect from excess liquids', 'Protect from loss of liquids', 'Protect from gases', 'Protect from wind', 'Protect from temperature', 'Protect from fire', 'Protect from ice', 'Protect from light', 'Protect from chemicals', 'Protect from radiation', 'Manage mechanical forces', 'Manage external forces', 'Manage impact', 'Regulate drag or turbulence', 'Manage stress or strain', 'Prevent buckling', 'Prevent fracture or rupture', 'Manage shear', 'Prevent or allow deformation', 'Prevent degradation', 'Prevent fatigue', 'Control erosion', 'Regulate wear', 'Sustain ecological community', 'Individual benefit', 'Regulate reproduction or growth', 'Self-replicate', 'Maintain homeostasis or equillibrium', 'Group benefit', 'Coordinate by self-organization', 'Cooperate within or between species', 'Compete within or between species', 'Manage environmental disturbances', 'Manage populations or habitats', 'Manage pests or diseases', 'Maintain biodiversity', 'Chemically assemble or break down', 'Chemically assemble', 'Chemically assemble inorganic compounds', 'Chemically assemble organic compounds', 'Self-assemble', 'Chemically break down', 'Chemically break down inorganic compounds', 'Chemically break down organic compounds', 'Sense send or process information', 'Send signals', 'Send light signals in the visible spectrum', 'Send light signals in the non-visible spectrum', 'Send sound signals', 'Send tactile signals', 'Send chemical signals', 'Send vibratory signals', 'Send electrical or magnetic signals', 'Process signals', 'Differentiate signal from noise', 'Convert signals', 'Respond to signals', 'Sense signals or environmental cues', 'Sense light in the visible spectrum', 'Sense light in the non-visible spectrum', 'Sense electricity or magnetism', 'Sense disease in a living system', 'Sense touch and mechanical forces', 'Sense chemicals', 'Sense atmospheric conditions', 'Sense sound or vibrations', 'Sense temperature cues', 'Sense motion', 'Sense spatial awareness or balance or orientation', 'Sense shape and or or pattern', 'Manipulate solids liquids gases or energy', 'Capture resources', 'Capture solids', 'Capture liquids', 'Capture gases', 'Capture energy', 'Absorb and or or filter resources', 'Absorb and or or filter solids', 'Absorb and or or filter liquids', 'Absorb and or or filter gases', 'Store resources', 'Store solids', 'Store liquids', 'Store gases', 'Store energy', 'Distribute or expel resources', 'Distribute or expel solids', 'Distribute or expel liquids', 'Distribute or expel gases', 'Distribute or expel energy', 'Detox or purify']
candidate_labels_leaves = []
candidate_labels_lvl1 = ['physically assembling or disassembling',
                         'protecting',
                         'sensing, sending or processing information',
                         'chemically assembling or breaking down',
                         'managing mechanical forces',
                         'attaching',
                         'movement',
                         'manipulating solids, liquids, gases or energy',
                         'sustaining ecological community']
candidate_labels_lvl1b = ['assembling',
                         'protecting',
                         'signals',
                         'chemical assembly',
                         'structural integrity',
                         'attaching',
                         'movement',
                         'purifying',
                         'ecological community']

candidate_labels_lvl1c = ['physically_assemble_or_disassemble',
                          'protect_from_harm',
                          'sense_send_or_process_information',
                          'chemically_modify_or_change_energy_state',
                          'maintain_structural_integrity',
                          'attach',
                          'move',
                          'process_resources',
                          'sustain_ecological_community',
                          'manipulate_solids_liquids_gases_or_energy',
                          'change_size_or_color']

In [70]:
classifier(sequence_to_classify, candidate_labels_lvl1, multi_label=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'labels': ['protecting',
  'manipulating solids, liquids, gases or energy',
  'managing mechanical forces',
  'movement',
  'sensing, sending or processing information',
  'sustaining ecological community',
  'physically assembling or disassembling',
  'chemically assembling or breaking down',
  'attaching'],
 'scores': [0.715483546257019,
  0.7145540118217468,
  0.7141457796096802,
  0.7139267325401306,
  0.7137396931648254,
  0.7133082747459412,
  0.7125199437141418,
  0.7119426727294922,
  0.7099123597145081],
 'sequence': 'Evidence for self-cleaning in gecko setae. A tokay gecko can cling to virtually any surface and support its body mass with a single toe by using the millions of keratinous setae on its toe pads. Each seta branches into hundreds of 200-nm spatulae that make intimate contact with a variety of surface profiles. We showed previously that the combined surface area of billions of spatulae maximizes van der Waals interactions to generate large adhesive and shear forces

In [71]:
classifier2 = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [103]:
classifier2(sequence_to_classify6, candidate_labels_lvl1c, multi_label=True)

{'labels': ['sense_send_or_process_information',
  'change_size_or_color',
  'maintain_structural_integrity',
  'move',
  'process_resources',
  'protect_from_harm',
  'attach',
  'sustain_ecological_community',
  'physically_assemble_or_disassemble',
  'manipulate_solids_liquids_gases_or_energy',
  'chemically_modify_or_change_energy_state'],
 'scores': [0.14060430228710175,
  0.10475803166627884,
  0.09246211498975754,
  0.09237729012966156,
  0.08839617669582367,
  0.07898588478565216,
  0.06887102127075195,
  0.0518636591732502,
  0.0395553894340992,
  0.03389647603034973,
  0.01379806362092495],
 'sequence': 'Autotomy refers to the voluntary shedding of a body part; a renowned example is tail loss among lizards as a response to attempted predation. Although many aspects of lizard tail autotomy have been studied, the detailed morphology and mechanism remains unclear. In the present study, we showed that tail shedding by the Tokay gecko (Gekko gecko) and the associated extracellular

In [ ]:
from sklearn.metrics import ndcg_score

